In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 20 23:51:40 2018

@author: abhishek
"""

import datetime
import nltk
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
#nltk.download('averaged_perceptron_tagger')
#nltk.download('punkt')
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split


from os import chdir
import datetime
import os
import gensim


C:\Users\abhim\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# df = pd.read_csv('Card_Transactions.csv') ;

df = pd.read_csv('res_purchase_card_(pcard)_fiscal_year_2014_3pcd-aiuu (1).csv') ;

df.shape

(442458, 11)

In [3]:
df.columns

Index(['Year-Month', 'Agency Number', 'Agency Name', 'Cardholder Last Name',
       'Cardholder First Initial', 'Description', 'Amount', 'Vendor',
       'Transaction Date', 'Posted Date', 'Merchant Category Code (MCC)'],
      dtype='object')

In [4]:
df.dtypes

Year-Month                        int64
Agency Number                     int64
Agency Name                      object
Cardholder Last Name             object
Cardholder First Initial         object
Description                      object
Amount                          float64
Vendor                           object
Transaction Date                 object
Posted Date                      object
Merchant Category Code (MCC)     object
dtype: object

In [37]:
# pd.to_datetime(df['Transaction Date'], format='%d%b%Y:%H:%M:%S.%f')

In [38]:
# df['date_type_check'] = len(str(df['Transaction Date']))

# df['date_type_check'].unique()

# set(df['date_type_check'])

# pd.to_datetime(df['Transaction Date'], format='%d%b%Y:%H:%M:%S.%f')

In [5]:
df.tail(10)

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC)
442448,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,GENERAL PURCHASE,2622.70,HAROLDS AUTO PARTS OF,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,AUTOMOTIVE PARTS AND ACCESSORIES STORES
442449,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,GENERAL PURCHASE,132790.14,EVANS ENTERPRISES,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,"COMMERCIAL EQUIPMENT, NOT ELSEWHERE CLASSIFIED"
442450,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,GENERAL PURCHASE,1307.58,YELLOWHOUSE MACHINERY CO-,06/19/2014 12:00:00 AM,06/23/2014 12:00:00 AM,"MISC. AUTOMOTIVE,AIRCRAFT,AND FARM EQUIPMENT D..."
442451,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,Payment on Account Each,3302.60,O REILLY AUTO 00050559,06/19/2014 12:00:00 AM,06/23/2014 12:00:00 AM,AUTOMOTIVE PARTS AND ACCESSORIES STORES
442452,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,FUEL PUMP EA|FUEL PUMP GASKET EA|CLAMP 5 EA|CO...,189.13,WHITE STAR MACHINERY,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,"COMMERCIAL EQUIPMENT, NOT ELSEWHERE CLASSIFIED"
442453,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,GENERAL PURCHASE,992.00,TOTAL VALVE SYSTEMS,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,HARDWARE EQUIPMENT AND SUPPLIES
442454,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,GENERAL PURCHASE,2415.00,PRYOR WASTE & RECYCLING L,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,"UTILITIES - ELECTRIC,GAS,WATER,SANITARY"
442455,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,GENERAL PURCHASE,177.50,ACCURATE ENVIROMENTAL LAB,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED
442456,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,GENERAL PURCHASE,32457.00,RED BUD SERVICE INC,06/19/2014 12:00:00 AM,06/23/2014 12:00:00 AM,PROFESSIONAL SERVICES NOT ELSEWHERE CLASSIFIED
442457,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,GENERAL PURCHASE,4714.07,WARRENCAT CREDIT DEPT,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,MISCELLANEOUS AND SPECIALTY RETAIL STORES


In [6]:
df['cardholdername'] = df['Cardholder Last Name'].map(str)+'-'+df['Cardholder First Initial'].map(str)

In [41]:
# df.dropna(subset=['Consumer complaint narrative'],inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442458 entries, 0 to 442457
Data columns (total 12 columns):
Year-Month                      442458 non-null int64
Agency Number                   442458 non-null int64
Agency Name                     442458 non-null object
Cardholder Last Name            442458 non-null object
Cardholder First Initial        442458 non-null object
Description                     442458 non-null object
Amount                          442458 non-null float64
Vendor                          442458 non-null object
Transaction Date                442458 non-null object
Posted Date                     442458 non-null object
Merchant Category Code (MCC)    442458 non-null object
cardholdername                  442458 non-null object
dtypes: float64(1), int64(2), object(9)
memory usage: 40.5+ MB


In [8]:
df.index

RangeIndex(start=0, stop=442458, step=1)

In [9]:
df.sort_values(by=['cardholdername','Transaction Date' ], inplace = True )

In [10]:
len(df.Vendor.unique())

86729

In [46]:
len(df.Vendor)

442458

In [47]:
type(set(df["Vendor"]))

set

In [48]:
# del df1

In [11]:
# def f(x):
#      return Series(dict(Vendor = "{%s}" % ', '.join(x['Vendor'])))

# df.groupby('cardholdername').apply(f)

df1 = df.groupby(['cardholdername'],as_index=False).agg(lambda Vendor : ';'.join(Vendor))

In [50]:
df1.columns

Index(['cardholdername', 'Agency Name', 'Cardholder Last Name',
       'Cardholder First Initial', 'Description', 'Vendor', 'Transaction Date',
       'Posted Date', 'Merchant Category Code (MCC)'],
      dtype='object')

In [13]:
df1.drop(['Description'], axis = 1, inplace = True)
df1.drop(['Agency Name'], axis = 1, inplace = True)
df1.drop(['Cardholder Last Name'], axis = 1, inplace = True)
df1.drop(['Cardholder First Initial'], axis = 1, inplace = True)
df1.drop(['Merchant Category Code (MCC)'], axis = 1, inplace = True)
df1.drop(['Transaction Date'], axis = 1, inplace = True)
df1.drop(['Posted Date'], axis = 1, inplace = True)

KeyError: "labels ['Description'] not contained in axis"

In [14]:
df1.shape

(5213, 2)

In [15]:
df1.head(50)

,cardholdername,Vendor
0,15000-1,PAYMENT ADJUSTMENT
1,24100-2,PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINAN...
2,42000-4,PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURC...
3,44600-4,CBR MAN CK
4,48000-4,LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE...
5,53000-5,PURCHASE FINANCE CHARGE
6,53100-5,PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINAN...
7,60500-6,LATE PAYMENT FEE;PURCHASE FINANCE CHARGE
8,75000-7,PAYMENT ADJUSTMENT;PAYMENT ADJUSTMENT
9,77000-7,FINANCE CHARGE ADJUSTMENT;FINANCE CHARGE ADJUS...


In [54]:
#only required for dislaying the comments completely on your screen
pd.options.display.max_colwidth =1000;

In [16]:
len(df['cardholdername'].unique())

5213

In [17]:
vendor_list = df1['Vendor'].tolist()

In [57]:
len(vendor_list)

5213

In [58]:
type(str(vendor_list[1]))

str

In [18]:
vendors_list = []
for vendors in vendor_list:
    vendors_list.append(str(vendors).split(";"))
    
    
    

# str(vendor_list[1]).split(";")[2]
# vendor_list[1]

In [19]:
len(vendors_list)

5213

In [20]:
vendors_list[1]

['PURCHASE  FINANCE CHARGE',
 'CREDIT PURCHASE FINANCE C',
 'PURCHASE  FINANCE CHARGE',
 'PURCHASE  FINANCE CHARGE',
 'LATE PAYMENT FEE',
 'PURCHASE  FINANCE CHARGE',
 'LATE PAYMENT FEE',
 'PURCHASE  FINANCE CHARGE',
 'PURCHASE  FINANCE CHARGE',
 'LATE PAYMENT FEE']

In [62]:
vendors_list[2][4]

'PURCHASE  FINANCE CHARGE'

In [21]:
model = gensim.models.Word2Vec(vendors_list,size=150,window=10,min_count=1, workers=10 )

In [22]:
model.train( vendors_list, total_examples=len(vendors_list), epochs=100)

(38549003, 44245800)

In [65]:
# save model
model.save('model_transactions.bin')

In [66]:
vendors = list(model.wv.vocab)

In [67]:
vendors[0]

'PAYMENT ADJUSTMENT'

In [68]:
model[vendors[0]]

C:\Users\abhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.1876203 ,  0.18114004,  0.07488787, -0.20532927,  0.6997193 ,
        0.53871816,  0.6552671 ,  0.47263938,  0.36206573,  0.23294884,
        0.17366548, -0.04762812,  0.49370918,  0.20677282, -0.18142283,
        0.27241194, -0.01541716,  0.30797878,  0.08819734, -0.52070856,
       -0.00732393, -0.47631264,  0.1675265 , -0.13079664,  0.49453637,
       -0.01574939, -0.62059146, -0.60965526, -0.01624569,  0.14431737,
        0.20967157, -0.03587123,  0.3388252 ,  0.28594694, -0.16743273,
       -0.17546023, -0.1692822 , -0.6008248 , -0.5139442 , -0.22588952,
       -0.29226723,  0.34986475,  0.6285293 , -0.13475595,  0.0153986 ,
       -0.25673872, -0.15035133, -0.05329133, -0.18555145, -0.53385687,
        0.05911183, -0.12146133,  0.19507673, -0.85679555,  0.3005053 ,
        0.5282212 , -0.14498676, -0.05068554,  0.02737937, -0.00266013,
        0.1396782 , -0.01599292, -0.03161122, -0.22844827, -0.5983683 ,
       -0.18400781,  0.30641052, -0.31498674,  0.22283801,  0.37

In [69]:
len(list(model.wv.vocab))

86729

In [70]:
list(model['CREDIT PURCHASE FINANCE C'])

C:\Users\abhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


[-0.8544998,
 -0.27135864,
 0.4448259,
 -0.73786044,
 0.66636056,
 0.28688776,
 0.48349684,
 0.46014684,
 0.6825309,
 0.2233471,
 0.022792242,
 -0.110113956,
 0.2474085,
 -0.3296741,
 -0.39641717,
 -0.18007566,
 0.23247372,
 -0.18702616,
 -0.19997767,
 -0.34845704,
 -0.083483465,
 0.20943543,
 0.26158735,
 -0.2203227,
 0.17770313,
 0.37154323,
 5.6548644e-05,
 -0.22909641,
 0.36484578,
 -0.33245626,
 -0.15957318,
 -0.025865046,
 0.177875,
 -0.09801817,
 0.38091362,
 -0.0016794672,
 -0.52984256,
 -0.21012525,
 -0.22239868,
 0.12202446,
 0.1875093,
 0.57054967,
 -0.19152947,
 0.045629077,
 0.116850786,
 -0.12625402,
 -0.27393505,
 -0.03797479,
 0.004429756,
 0.60272795,
 0.02164839,
 -0.5339395,
 0.12389846,
 -0.46735197,
 0.17216618,
 0.056727972,
 0.06485856,
 -0.106257446,
 0.4864692,
 -0.022674302,
 0.38895792,
 0.0010214037,
 0.25786644,
 -0.14706603,
 -0.05340497,
 -0.027104111,
 -0.07474898,
 -0.056645527,
 0.45359632,
 0.011999992,
 0.122864604,
 -0.49966356,
 0.018670607,
 -0.01

In [71]:
list_word2vec = []
# for vendor in vendors:
#     list_vec = []
#     list_word2vec.append(vendor)
#     list(model['THE CHRONICLE'])
# model[vendors[0]]

# for i in range(len(vendors)):
#      list_word2vec[i][0] = vendors[i]

test = np.array(model[vendors[8358]])[1];
test
        
    
    
    

C:\Users\abhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


0.13204165

In [72]:
len(vendors)

86729

In [73]:
result_array = np.empty((8359, 150));
# result_array[8358][149];
# result_array.shape[0]

for i in range(result_array.shape[0]):
    for j in range(result_array.shape[1]):
        result_array[i][j] = np.array(model[vendors[i]])[j]
        



# for line in data_array:
#         result_array = np.append(result_array, [result], axis=0)

C:\Users\abhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [74]:
result_array[0][1]

0.18114003539085388

In [75]:
vectors = pd.DataFrame(result_array)

In [76]:
df_vendors = pd.DataFrame(np.array(vendors))

In [77]:
vectors.shape

(8359, 150)

In [78]:
vendors[5]

'CREDIT LATE CHARGE FEE'

In [79]:
df_vendors.shape

(86729, 1)

In [80]:
df_vendors.columns = ['vendor_name']

In [81]:
df_vendor_vec = pd.concat([df_vendors, vectors], axis = 1)

In [82]:
df_vendor_vec.head()

,vendor_name,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,PAYMENT ADJUSTMENT,-0.187620,0.181140,0.074888,-0.205329,0.699719,0.538718,0.655267,0.472639,0.362066,...,0.113732,-0.155741,-0.101212,-0.628485,0.355282,0.230777,-0.211435,-0.109958,0.601161,-0.140754
1,PURCHASE FINANCE CHARGE,-3.227230,-1.126077,1.601228,-2.858145,2.606008,1.029714,1.668809,1.758405,2.580479,...,0.233532,1.341058,-1.037389,-0.523728,0.726783,1.981835,0.592565,-1.212989,1.057108,-1.876818
2,CREDIT PURCHASE FINANCE C,-0.854500,-0.271359,0.444826,-0.737860,0.666361,0.286888,0.483497,0.460147,0.682531,...,0.046031,0.350452,-0.289700,-0.180288,0.203968,0.473718,0.173896,-0.302755,0.239429,-0.520584
3,LATE PAYMENT FEE,-2.369387,-0.788545,1.159640,-2.129657,1.967685,0.820172,1.292833,1.328750,1.924593,...,0.176931,0.999069,-0.748713,-0.425342,0.559287,1.461481,0.436135,-0.901660,0.791876,-1.391317
4,CBR MAN CK,-0.109160,-0.039830,0.075329,-0.157691,0.274772,0.188879,0.255027,0.177474,0.199276,...,0.066100,-0.026717,-0.069987,-0.169633,0.038311,0.145400,-0.046362,-0.087631,0.202173,-0.091267


In [83]:
df_master_vectors = pd.merge(df, df_vendor_vec, how='inner', left_on=['Vendor'], right_on=['vendor_name'])

In [84]:
df_master_vectors.head(2)

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,...,140,141,142,143,144,145,146,147,148,149
0,201404,15000,UNIV.OF SCIENCE & ARTS OF OK,15000,1,GENERAL PURCHASE,4626.46,PAYMENT ADJUSTMENT,03/10/2014 12:00:00 AM,04/07/2014 12:00:00 AM,...,0.113732,-0.155741,-0.101212,-0.628485,0.355282,0.230777,-0.211435,-0.109958,0.601161,-0.140754
1,201309,75000,TULSA COMMUNITY COLLEGE,75000,7,GENERAL PURCHASE,29728.88,PAYMENT ADJUSTMENT,08/15/2013 12:00:00 AM,09/30/2013 12:00:00 AM,...,0.113732,-0.155741,-0.101212,-0.628485,0.355282,0.230777,-0.211435,-0.109958,0.601161,-0.140754


In [85]:
df_master_vectors.shape

(442458, 163)

In [86]:
df1.head()

,cardholdername,Vendor
0,15000-1,PAYMENT ADJUSTMENT
1,24100-2,PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINANCE C;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE
2,42000-4,PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE
3,44600-4,CBR MAN CK
4,48000-4,LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;CREDIT LATE CHARGE FEE;PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINANCE C;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE


In [87]:
df_master_vectors.columns[13]

0

In [88]:
df_master_vectors.iloc[1:2,0:15]

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC),cardholdername,vendor_name,0,1
1,201309,75000,TULSA COMMUNITY COLLEGE,75000,7,GENERAL PURCHASE,29728.88,PAYMENT ADJUSTMENT,08/15/2013 12:00:00 AM,09/30/2013 12:00:00 AM,OTHER FEES,75000-7,PAYMENT ADJUSTMENT,-0.18762,0.18114


In [89]:
df_master_vectors.columns

Index([              'Year-Month',            'Agency Number',
                    'Agency Name',     'Cardholder Last Name',
       'Cardholder First Initial',              'Description',
                         'Amount',                   'Vendor',
               'Transaction Date',              'Posted Date',
       ...
                              140,                        141,
                              142,                        143,
                              144,                        145,
                              146,                        147,
                              148,                        149],
      dtype='object', length=163)

In [90]:
df_master_vectors.columns = ['Year-Month', 'Agency Number','Agency Name','Cardholder Last Name','Cardholder First Initial','Description','Amount',
'Vendor','Transaction Date','Posted Date','Merchant Category Code (MCC)', 'cardholdername','vendor_name',
'vector_0','vector_1','vector_2','vector_3','vector_4','vector_5','vector_6','vector_7','vector_8','vector_9',
'vector_10','vector_11',
'vector_12','vector_13',
'vector_14','vector_15',
'vector_16','vector_17',
'vector_18','vector_19',
'vector_20','vector_21',
'vector_22','vector_23',
'vector_24','vector_25',
'vector_26','vector_27',
'vector_28','vector_29',
'vector_30','vector_31',
'vector_32','vector_33',
'vector_34','vector_35',
'vector_36','vector_37',
'vector_38','vector_39',
'vector_40','vector_41',
'vector_42','vector_43',
'vector_44','vector_45',
'vector_46','vector_47',
'vector_48','vector_49',
'vector_50','vector_51',
'vector_52','vector_53',
'vector_54','vector_55',
'vector_56','vector_57',
'vector_58','vector_59',
'vector_60','vector_61',
'vector_62','vector_63',
'vector_64','vector_65',
'vector_66','vector_67',
'vector_68','vector_69',
'vector_70','vector_71',
'vector_72','vector_73',
'vector_74','vector_75',
'vector_76','vector_77',
'vector_78','vector_79',
'vector_80','vector_81',
'vector_82','vector_83',
'vector_84','vector_85',
'vector_86',
'vector_87','vector_88',
'vector_89','vector_90',
'vector_91','vector_92',
'vector_93','vector_94',
'vector_95','vector_96',
'vector_97','vector_98',
'vector_99','vector_100',
'vector_101','vector_102',
'vector_103','vector_104',
'vector_105','vector_106',
'vector_107','vector_108',
'vector_109','vector_110',
'vector_111','vector_112',
'vector_113','vector_114','vector_115',
'vector_116','vector_117','vector_118',
'vector_119','vector_120',
'vector_121','vector_122',
'vector_123','vector_124',
'vector_125','vector_126',
'vector_127','vector_128',
'vector_129','vector_130',
'vector_131','vector_132',
'vector_133','vector_134',
'vector_135','vector_136',
'vector_137','vector_138',
'vector_139','vector_140',
'vector_141','vector_142',
'vector_143','vector_144',
'vector_145','vector_146',
'vector_147','vector_148',
'vector_149']

In [91]:
pd.DataFrame(df_master_vectors["".join(["vector_", "1" ])].groupby([df_master_vectors['cardholdername']]).mean())

,vector_1
cardholdername,
15000-1,0.181140
24100-2,-0.939346
42000-4,-0.978407
44600-4,-0.039830
48000-4,-0.830083
53000-5,-1.126077
53100-5,-0.698718
60500-6,-0.957311
75000-7,0.181140


In [92]:
"".join(["vector_", "1" ])

'vector_1'

In [93]:
# Group the data frame by month and item and extract a number of stats from each group
for i in range(150):
    temp_df = pd.DataFrame(df_master_vectors["".join(["vector_", str(i) ])].groupby([df_master_vectors['cardholdername']]).mean())
    df1 = pd.merge(df1, temp_df, how='inner', left_on=['cardholdername'], right_on=['cardholdername'])
    del temp_df

In [94]:
df1.head(2)

,cardholdername,Vendor,vector_0,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,...,vector_140,vector_141,vector_142,vector_143,vector_144,vector_145,vector_146,vector_147,vector_148,vector_149
0,15000-1,PAYMENT ADJUSTMENT,-0.187620,0.181140,0.074888,-0.205329,0.699719,0.538718,0.655267,0.472639,...,0.113732,-0.155741,-0.101212,-0.628485,0.355282,0.230777,-0.211435,-0.109958,0.601161,-0.140754
1,24100-2,PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINANCE C;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE,-2.732604,-0.939346,1.353111,-2.427570,2.220546,0.892568,1.437485,1.499683,...,0.197802,1.139400,-0.876017,-0.459868,0.624253,1.674917,0.503769,-1.028567,0.895770,-1.595544


In [95]:
df1.shape

(5213, 152)

In [96]:
df_vendor_vec.head(2)

,vendor_name,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,PAYMENT ADJUSTMENT,-0.18762,0.181140,0.074888,-0.205329,0.699719,0.538718,0.655267,0.472639,0.362066,...,0.113732,-0.155741,-0.101212,-0.628485,0.355282,0.230777,-0.211435,-0.109958,0.601161,-0.140754
1,PURCHASE FINANCE CHARGE,-3.22723,-1.126077,1.601228,-2.858145,2.606008,1.029714,1.668809,1.758405,2.580479,...,0.233532,1.341058,-1.037389,-0.523728,0.726783,1.981835,0.592565,-1.212989,1.057108,-1.876818


In [97]:
df_vendor_vec.shape

(86729, 151)

In [98]:
df_vendor_vec.columns

Index(['vendor_name',             0,             1,             2,
                   3,             4,             5,             6,
                   7,             8,
       ...
                 140,           141,           142,           143,
                 144,           145,           146,           147,
                 148,           149],
      dtype='object', length=151)

In [99]:
np.array(df_vendor_vec)[1][1:151].shape

(150,)

In [100]:
np.array(df1)[0][2:152].shape

(150,)

In [101]:
def cos_sim(a, b):
 dot_product = np.dot(a, b)
 norm_a = np.linalg.norm(a)
 norm_b = np.linalg.norm(b)
 return dot_product / (norm_a * norm_b)

In [102]:
df1.columns

Index(['cardholdername', 'Vendor', 'vector_0', 'vector_1', 'vector_2',
       'vector_3', 'vector_4', 'vector_5', 'vector_6', 'vector_7',
       ...
       'vector_140', 'vector_141', 'vector_142', 'vector_143', 'vector_144',
       'vector_145', 'vector_146', 'vector_147', 'vector_148', 'vector_149'],
      dtype='object', length=152)

In [103]:
# cos_sim_list = []
# for i in range(df1.shape[0]):
#     temp_list = []
#     for j in range(df_vendor_vec.shape[0]):        
#         temp_list.append(cos_sim(np.array(df_vendor_vec)[j][1:], np.array(df1)[i][2]))
        
#     sorted_temp_list_index = sorted(range(len(temp_list)), key=lambda k: temp_list[k])
#     sorted_temp_list_index.reverse()
#     temp_list.sort(reverse=True)   
    
#     df1['max_cos_sim_1'][i] = temp_list[0]
#     df1['vendor_1'][i] = df_vendor_vec[sorted_temp_list_index[0]]    
#     df1['max_cos_sim_2'][i] = temp_list[1]
#     df1['vendor_2'][i] = df_vendor_vec[sorted_temp_list_index[1]]
#     df1['max_cos_sim_3'][i] = temp_list[2]
#     df1['vendor_3'][i] = df_vendor_vec[sorted_temp_list_index[2]]
#     df1['max_cos_sim_4'][i] = temp_list[3]
#     df1['vendor_4'][i] = df_vendor_vec[sorted_temp_list_index[3]]
#     df1['max_cos_sim_5'][i] = temp_list[4]
#     df1['vendor_5'][i] = df_vendor_vec[sorted_temp_list_index[4]]

# #     df1['max_cos_sim'][i] = max(temp_list)
# #     df1['max_cos_sim'][i] = max(cos_sim_list.append(cos_sim(np.array(df_vendor_vec)[i][1:151], np.array(df1)[i][2:152])))
    
    

In [107]:
df1.head(20)

,cardholdername,Vendor,vector_0,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,...,max_cos_sim_15,sim_vendor_15,max_cos_sim_16,sim_vendor_16,max_cos_sim_17,sim_vendor_17,max_cos_sim_18,sim_vendor_18,max_cos_sim_19,sim_vendor_19
0,15000-1,PAYMENT ADJUSTMENT,-0.187620,0.181140,0.074888,-0.205329,0.699719,0.538718,0.655267,0.472639,...,,,,,,,,,,
1,24100-2,PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINANCE C;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE,-2.732604,-0.939346,1.353111,-2.427570,2.220546,0.892568,1.437485,1.499683,...,,,,,,,,,,
2,42000-4,PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE,-2.851924,-0.978407,1.408033,-2.539431,2.326742,0.938039,1.504319,1.570431,...,,,,,,,,,,
3,44600-4,CBR MAN CK,-0.109160,-0.039830,0.075329,-0.157691,0.274772,0.188879,0.255027,0.177474,...,,,,,,,,,,
4,48000-4,LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;CREDIT LATE CHARGE FEE;PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINANCE C;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE;LATE PAYMENT FEE;LATE PAYMENT FEE;PURCHASE FINANCE CHARGE,-2.438016,-0.830083,1.204111,-2.170784,1.991907,0.810057,1.295371,1.347466,...,,,,,,,,,,
5,53000-5,PURCHASE FINANCE CHARGE,-3.227230,-1.126077,1.601228,-2.858145,2.606008,1.029714,1.668809,1.758405,...,,,,,,,,,,
6,53100-5,PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINANCE C;PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINANCE C;PURCHASE FINANCE CHARGE;CREDIT PURCHASE FINANCE C,-2.040865,-0.698718,1.023027,-1.798003,1.636184,0.658301,1.076153,1.109276,...,,,,,,,,,,
7,60500-6,LATE PAYMENT FEE;PURCHASE FINANCE CHARGE,-2.798309,-0.957311,1.380434,-2.493901,2.286847,0.924943,1.480821,1.543577,...,,,,,,,,,,
8,75000-7,PAYMENT ADJUSTMENT;PAYMENT ADJUSTMENT,-0.187620,0.181140,0.074888,-0.205329,0.699719,0.538718,0.655267,0.472639,...,,,,,,,,,,
9,77000-7,FINANCE CHARGE ADJUSTMENT;FINANCE CHARGE ADJUSTMENT;DEBIT PURCH FINANCE CHG;DEBIT PURCH FINANCE CHG;PAYMENT ADJUSTMENT;PAYMENT ADJUSTMENT,-0.174236,0.158107,0.080898,-0.196350,0.627383,0.493030,0.604984,0.435645,...,,,,,,,,,,


In [105]:
# #trial
# cos_sim_list = []
# reach = 5
# for l in range(reach):
#     df1['max_cos_sim_'+ str(l)] = ''
#     df1['sim_vendor_'+ str(l) ] = ''  
    
    
# for i in range(df1.shape[0]):
#     temp_list = []
#     print(i)
    
#     for j in range(df_vendor_vec.shape[0]): 
#         print(j)
#         temp_list.append(cos_sim(np.array(df_vendor_vec)[j][1:151], np.array(df1)[i][2:152]))
# #         print(temp_list)
#         print(j)
    
# #     print(temp_list[0:4])        
#     sorted_temp_list_index = sorted(range(len(temp_list)), key=lambda k: temp_list[k])
#     sorted_temp_list_index.reverse()
#     temp_list.sort(reverse=True)
#     for l in range(reach):
#         df1['max_cos_sim_'+ str(l)][i] = temp_list[l]
# #         df1['sim_vendor_'+ str(l) ][i] = df_vendor_vec['vendor_name'][sorted_temp_list_index[l]]
#     print(i)   
    
#     df1['max_cos_sim_1'][i] = temp_list[0]
#     df1['vendor_1'][i] = df_vendor_vec[sorted_temp_list_index[0]]    
#     df1['max_cos_sim_2'][i] = temp_list[1]
#     df1['vendor_2'][i] = df_vendor_vec[sorted_temp_list_index[1]]
#     df1['max_cos_sim_3'][i] = temp_list[2]
#     df1['vendor_3'][i] = df_vendor_vec[sorted_temp_list_index[2]]
#     df1['max_cos_sim_4'][i] = temp_list[3]
#     df1['vendor_4'][i] = df_vendor_vec[sorted_temp_list_index[3]]
#     df1['max_cos_sim_5'][i] = temp_list[4]
#     df1['vendor_5'][i] = df_vendor_vec[sorted_temp_list_index[4]]

In [108]:
#final

cos_sim_list = []
reach = 20
for l in range(reach):
    df1['max_cos_sim_'+ str(l)] = ''
    df1['sim_vendor_'+ str(l) ] = ''  
    
    
for i in range(25):
    temp_list = []
    print(i)
    
    for j in range(200): 
        print(j)
        temp_list.append(cos_sim(np.array(df_vendor_vec)[j][1:151], np.array(df1)[i][2:152]))
#         print(temp_list)
        print(j)
    
#     print(temp_list[0:4])        
    sorted_temp_list_index = sorted(range(len(temp_list)), key=lambda k: temp_list[k])
    sorted_temp_list_index.reverse()
    temp_list.sort(reverse=True)
    for l in range(reach):
        df1['max_cos_sim_'+ str(l)][i] = temp_list[l]
        df1['sim_vendor_'+ str(l) ][i] = df_vendor_vec['vendor_name'][sorted_temp_list_index[l]]
    print(i)   
    
#     df1['max_cos_sim_1'][i] = temp_list[0]
#     df1['vendor_1'][i] = df_vendor_vec[sorted_temp_list_index[0]]    
#     df1['max_cos_sim_2'][i] = temp_list[1]
#     df1['vendor_2'][i] = df_vendor_vec[sorted_temp_list_index[1]]
#     df1['max_cos_sim_3'][i] = temp_list[2]
#     df1['vendor_3'][i] = df_vendor_vec[sorted_temp_list_index[2]]
#     df1['max_cos_sim_4'][i] = temp_list[3]
#     df1['vendor_4'][i] = df_vendor_vec[sorted_temp_list_index[3]]
#     df1['max_cos_sim_5'][i] = temp_list[4]
#     df1['vendor_5'][i] = df_vendor_vec[sorted_temp_list_index[4]]

0
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
15

C:\Users\abhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\abhim\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0
1
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151


186
187
187
188
188
189
189
190
190
191
191
192
192
193
193
194
194
195
195
196
196
197
197
198
198
199
199
6
7
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138


172
173
173
174
174
175
175
176
176
177
177
178
178
179
179
180
180
181
181
182
182
183
183
184
184
185
185
186
186
187
187
188
188
189
189
190
190
191
191
192
192
193
193
194
194
195
195
196
196
197
197
198
198
199
199
12
13
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
12

157
158
158
159
159
160
160
161
161
162
162
163
163
164
164
165
165
166
166
167
167
168
168
169
169
170
170
171
171
172
172
173
173
174
174
175
175
176
176
177
177
178
178
179
179
180
180
181
181
182
182
183
183
184
184
185
185
186
186
187
187
188
188
189
189
190
190
191
191
192
192
193
193
194
194
195
195
196
196
197
197
198
198
199
199
18
19
0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
10

142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152
152
153
153
154
154
155
155
156
156
157
157
158
158
159
159
160
160
161
161
162
162
163
163
164
164
165
165
166
166
167
167
168
168
169
169
170
170
171
171
172
172
173
173
174
174
175
175
176
176
177
177
178
178
179
179
180
180
181
181
182
182
183
183
184
184
185
185
186
186
187
187
188
188
189
189
190
190
191
191
192
192
193
193
194
194
195
195
196
196
197
197
198
198
199
199
24


In [ ]:
df1.head(5)

In [109]:
df1.to_csv("User_recommendations.csv")

In [ ]:
cos_sim(np.array(df_vendor_vec)[2][1:151], np.array(df1)[1][2:152])

In [ ]:
s = [2, 3, 1, 4, 5,1]
s_sorted_index = sorted(range(len(s)), key=lambda k: s[k])

In [ ]:
s_sorted_index

In [ ]:
s_sorted_index.reverse()

In [ ]:
s = [2, 3, 1, 4, 5,1]
s.sort(reverse=True)

In [ ]:
s

In [ ]:
s_sorted_index

In [ ]:
df1.shape

In [ ]:
df1.head()

In [ ]:
temp_df.head()

In [ ]:
df_master_vectors.columns[13]

In [ ]:
df_vendor_vec.head()

In [ ]:
df_vendor_vec.shape

In [ ]:
df['Vendor'].unique().shape

In [ ]:
w1 = 'BUDGET RENT-A-CAR'
model.wv.most_similar (positive = w1, topn=20)

In [ ]:
X = model[model.wv.vocab]

In [ ]:
from sklearn.decomposition import PCA
# from sklearn.decomposition import PCA
from matplotlib import pyplot

pca = PCA(n_components=2)
result = pca.fit_transform(X)

In [ ]:
pyplot.scatter(result[:, 0], result[:, 1])

In [ ]:
vendors = list(model.wv.vocab)
for i, vendor in enumerate(vendors):
    pyplot.annotate(vendor, xy=(result[i, 0], result[i, 1]))

In [ ]:
print(model['THE CHRONICLE'])

In [ ]:
type(model)

In [ ]:
w1 = 'THE CHRONICLE'
model.wv.most_similar (positive = w1, topn=20)